# Example Analysis Code

What follows are some basic analyses of the Duke Forge/SSRI COVID19 Digital Lab Social Distancing Survey Weeks 1 & 2.

Because this is a *weighted* survey, note that simple summary statistics will not accurately reflect population statistics. This notebook includes code that can be easily modified for calculating basically statistics using these weights. 

Also note that many variables have top-codes of `9`. This is because surveys were conducted with automated calling, so respondents had to punch in values on their phones. Thus be careful in interpreting top-values. 

The code the follows is written in Python and requires both the `pandas` and `statsmodels` packages, both of which can easily be installed using `conda` or `pip`. 

R users who wish to do their own analyses will likely find the `Survey` package to be of use. 

Note this data includes ALL responses, including partials. As a result, the composition of your sample may change if you start using demographic controls that occur later in the survey (and thus are less likely to be included for a given respondent). 

## Replicating Number of Children Per Household

Before analysis, we being by replicating one of the summary statistics provided by the firm that conducted this survey. Top-line summary statistics from the survey firm can be found in `40_reports/Survey_Summaries`. 

In particular, we'll replicate the share of households with No Children, One Child, and Two Children. The survey firm has reported that, after adjusting for weights, the proportion of households of each type are: 

- No Children: 60\% (Week 1), 65\% (Week 2)
- One Child: 15\% (Week 1), 13\% (Week 2)
- Two Children: 17\% (Week 1), 15\% (Week 2).

In [1]:
# Load the cleaned, and labeled version 
# of the survey. 
# The code that generates this cleaned data
# can be found in `10_import_and_format_week1.py`. 

import pandas as pd
import numpy as np

svy = pd.read_csv('../20_analysis_datasets/'
                  'merged_surveys.csv')
svy.columns

Index(['uniqueID', 'Date', 'Voter File Match',
       'Registered Voter (of Voter File Matches)', 'weight',
       'Q1. Health Quality', 'age', 'DEMOGRAPHICS - GENDER',
       'Q4. Number of People in HH', 'Q5. Children in HH',
       'Q6. Non-HH Face to Face Count', 'Q7. Six Feet Away? (If Q6 > 0)',
       'Q8. HH Member Going to Work',
       'Q9. Children Interacting with Other Children ',
       'Q10. Times in Group > 20 in Last Week', 'Family', 'Friends',
       'Co-workers', 'Clients, patients, or patrons',
       'Any other type of person not already mentioned',
       'Q12. Handwashing Count',
       'Q13. Currently Practicing Social Distancing?',
       'Q14. Currently Experiencing Symptoms?',
       'Q15. Likelihood of getting Coronavirus',
       'Q16. NC Response to Coronavirus', 'Q17. Changes to Routine ',
       'Q18. College Degree', 'Q19. Latino', 'Q20. Race',
       'Q21. Panel Willingness', 'Q19-20. Race + Ethnicity', 'Survey Mode',
       'DEMOGRAPHICS - RACE ON FILE

Now let's add a few convenience vars: 

In [2]:
svy['week1'] = svy['week'] == 'week1'
svy['week2'] = svy['week'] == 'week2'
svy['full_sample'] = 1

Note that all variables up to Q21 are questions asked in this survey. Subsequent variables come largely from Clarity Campaigns, which has done its best to match survey respondents with an internal database built off other sources. Information on these variables can be found in `40_reports`. 

In [3]:
len(svy)

3513

Now we can replicate results from Clarity. For this we will rely on the `DescrStatsW` function from `statsmodels`. [Documentation can be found here](https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.DescrStatsW.html), but the basic idea is that it takes a vector of data, a vector of weights, and then returns various weighted statistics. 

In [4]:
from statsmodels.stats.weightstats import DescrStatsW

for week in ['week1', 'week2']:
    
    oneweek = svy[svy['week'] == week].copy()
    for num in ['None', 'One', 'Two']:
        oneweek[num] = oneweek['Q5. Children in HH'] == num
        temp = oneweek[pd.notnull(oneweek['Q5. Children in HH'])]    
        w = DescrStatsW(temp[num], temp['weight'])
        print(f"Share of households with {num} kids in {week} is {w.mean:.1%}")

Share of households with None kids in week1 is 60.3%
Share of households with One kids in week1 is 14.8%
Share of households with Two kids in week1 is 17.1%
Share of households with None kids in week2 is 64.9%
Share of households with One kids in week2 is 12.8%
Share of households with Two kids in week2 is 14.5%


Of course, calculating overall averages is only kinda interesting. Generally, we want to know statistics for sub-populations. Normally, we'd just do this using the `groupby` operator, but things are a little more complicated with weighting. 

To begin, let's again calculate the proportion of households with different numbers of children using groupby for the trivial case where everyone is in the same group. Once we know that works, we can start looking at more interesting sub-populations. 

In [5]:
def get_group_mean(data, question):
    temp = data[[question, 'weight']]
    temp = temp[pd.notnull(temp[question])]
    wsvy = DescrStatsW(temp[question], temp['weight'])
    return wsvy.mean

for num in ['None', 'One', 'Two']:
    week1 = svy[svy['week'] == 'week1'].copy()
    week1[num] = (week1['Q5. Children in HH'] == num)
    week1 = week1[pd.notnull(week1['Q5. Children in HH'])]    

    week1['dummy'] = 1

    raw = week1[num].mean()

    w = week1.groupby('dummy').apply(lambda x: get_group_mean(x, num))
    w = w.iloc[0]
    
    print(f'Share households with {num} kids in Week 1: {w:.1%}')

Share households with None kids in Week 1: 60.3%
Share households with One kids in Week 1: 14.8%
Share households with Two kids in Week 1: 17.1%


Great! Now let's break down number of children by race. To do this, we'll first re-code race since most categories are too small to be statistically valuable:

In [6]:
# Values before grouping
race = 'Q19-20. Race + Ethnicity'
svy[race].value_counts(dropna=False)

White                 2525
Black                  751
Another race           104
Hispanic or Latino      92
Asian                   41
Name: Q19-20. Race + Ethnicity, dtype: int64

In [7]:
svy[race] = svy[race].replace({'Asian': 'Other',
                               'Hispanic or Latino': 'Other',
                               'Another race': 'Other'})
svy[race].value_counts(dropna=False)

White    2525
Black     751
Other     237
Name: Q19-20. Race + Ethnicity, dtype: int64

Now we can look at number of children by racial group:

In [8]:
svy['None'] = svy['Q5. Children in HH'] == 'None'
temp = svy[pd.notnull(svy['Q5. Children in HH'])]
svy.groupby(race).apply(lambda x: get_group_mean(x, 'None'))

Q19-20. Race + Ethnicity
Black    0.571685
Other    0.539243
White    0.597149
dtype: float64

## Analyzing COVID-Related Outcomes

Now that we've gotten the basics of working with weighted survey data out of the way (and by comparing our calculated values to known outcomes from the survey firm, we know we've done it correctly), let's start looking at some COVID-related variables!

### Large Groups

A key question in our analysis is whether people have been in large groups in the last week. Note that because this survey was conducted on the 29th, 30th, and 31st, most North Carolineans were not yet under a "shelter-in-place" order during the week preceding this survey, so we wouldn't expect people's answers to "How many times have you been in a group of > 20 people in the last week" to be all zeros!

Note that `9` is a top-code here, so values of `9` mean "9 or greater".

In [9]:
# Get weighted proportions in each category
big_group = 'Q10. Times in Group > 20 in Last Week'

for week in ['week1', 'week2']:
    def get_group_sumweights(data, question):
        temp = data[[question, 'weight']]
        temp = temp[pd.notnull(temp[question])]
        wsvy = DescrStatsW(temp[question], temp['weight'])
        return wsvy.sum_weights

    temp = svy[svy['week'] == week]
    sums = temp.groupby(big_group).apply(lambda x: get_group_sumweights(x, big_group))
    proportions = sums / sums.sum()
    print(f'In {week}, the number of (reported) times people had been in groups > 20 was:')
    print(proportions)
    print('\n')

In week1, the number of (reported) times people had been in groups > 20 was:
Q10. Times in Group > 20 in Last Week
0.0    0.787501
1.0    0.089631
2.0    0.040359
3.0    0.019480
4.0    0.004135
5.0    0.013134
6.0    0.004822
7.0    0.001940
8.0    0.000546
9.0    0.038451
dtype: float64


In week2, the number of (reported) times people had been in groups > 20 was:
Q10. Times in Group > 20 in Last Week
0.0    0.823140
1.0    0.088225
2.0    0.034875
3.0    0.011741
4.0    0.002627
5.0    0.005282
6.0    0.009145
7.0    0.000508
8.0    0.000996
9.0    0.023459
dtype: float64




So clearly the *vast* majority of people haven't been in big groups (or won't admit to it). So let's just look at the share of people who've EVER been in a big group in the last year, and see how it breaks down by sub-population. 

In [10]:
svy['any_group']= (svy[big_group] > 0) & pd.notnull(svy[big_group])
svy.loc[pd.isnull(svy[big_group]), 'any_group'] = np.nan

In [12]:
for week in ['week1', 'week2']:
    print(f'share ever a large group in week {week}')
    print(f"{svy[svy[week]].groupby('full_sample').apply(lambda x: get_group_mean(x, 'any_group')).iloc[0]:.1%}")
    print('\n')

share ever a large group in week week1
21.2%


share ever a large group in week week2
17.7%




In [13]:
# And by race
for week in ['week1', 'week2']:
    print(f'share ever a large group in week {week}')
    print(f"{svy[svy.week == week].groupby(race).apply(lambda x: get_group_mean(x, 'any_group'))}")
    print('\n')

share ever a large group in week week1
Q19-20. Race + Ethnicity
Black    0.240851
Other    0.264728
White    0.196275
dtype: float64


share ever a large group in week week2
Q19-20. Race + Ethnicity
Black    0.274025
Other    0.181535
White    0.151264
dtype: float64




Interestingly, this suggests a significant decline in groups OTHER than `Black`. 

In [14]:
race = 'Q19-20. Race + Ethnicity'
avg_num = svy.groupby(race).apply(lambda x: get_group_mean(x, 'any_group'))
avg_num

Q19-20. Race + Ethnicity
Black    0.259548
Other    0.223341
White    0.171468
dtype: float64

Some small differences. Clarity provides a "likelihood people attend church" we can check to see if that's driving things (Black North Carolineans have slightly higher likelihood of attending church, says Clarity data):

In [15]:
svy.groupby(race).apply(
    lambda x: get_group_mean(x, 'CHURCH ATTENDANCE'))

Q19-20. Race + Ethnicity
Black    3.428091
Other    2.616084
White    3.017049
dtype: float64

We can also look for variation in likelihood by education -- appears those without college degree more likely to be in groups. 

In [16]:
educ = 'Q18. College Degree'
avg_num = svy.groupby(educ).apply(
    lambda x: get_group_mean(x, 'any_group'))
avg_num

Q18. College Degree
4         [0.0]
No     0.255593
Yes    0.154899
dtype: object

## Distancing in last 24 hours

The survey also asks about number of people with whom one has had face-to-face interactions in the last 24 hours, then in how many of those interactions was the respondent able to maintain social distance. The difference is num of people they weren't able to keep distance with. 

However, note top-codes make interpreting this a little tricky...

In [17]:
svy['close_interacts'] = (svy['Q6. Non-HH Face to Face Count'] - 
                          svy['Q7. Six Feet Away? (If Q6 > 0)'])
svy.loc[svy['Q6. Non-HH Face to Face Count'] == 0, 'close_interacts'] = 0
svy.loc[svy['Q6. Non-HH Face to Face Count'] == 9, 'close_interacts'] = np.nan

In [18]:
svy.close_interacts.value_counts()

 0.0    1971
 1.0     289
 2.0     157
 3.0      78
 4.0      35
-1.0      32
 5.0      20
-3.0      13
 6.0      11
-7.0       9
-8.0       9
-2.0       8
-6.0       8
-4.0       7
 7.0       7
-5.0       7
 8.0       3
Name: close_interacts, dtype: int64

OK, negatives are clearly junk...

In [19]:
svy.loc[svy['close_interacts'] < 0, 'close_interacts'] = 0

In [20]:
sums = svy.groupby('close_interacts').apply(
           lambda x: get_group_sumweights(x, 'close_interacts'))
proportions = sums / sums.sum()
proportions

close_interacts
0.0    0.787248
1.0    0.093663
2.0    0.064381
3.0    0.025122
4.0    0.012273
5.0    0.006736
6.0    0.003029
7.0    0.004234
8.0    0.003314
dtype: float64

So 11% said they had at least one interaction without distancing. 